# Short Project 1 Final Report
### Kaitlyn Jacobs, Paolo Marra-Biggs, Nick Glaser
## Q1:
We chose the bacteria Saccharomyces cerevisiae, brewer’s yeast, because I was enjoying a cold one when picking the bacteria.  It’s a highly occurring bacterium not only in baking and brewing, but also in the functioning of the gut.  We found a collection of mutations to align to the genome that was sequenced on an Illumina HiSeq X Ten.
## Q2:
[Full Reference Genome](https://www.ncbi.nlm.nih.gov/genome/15?genome_assembly_id=22535)

[SRA Database Source](https://www.ncbi.nlm.nih.gov/sra/SRX6900124[accn])

## Q3:
We chose to use BWA and Bowtie2 as our aligners, and the index/mem (BWA) parameter space.  BWA gave us a 93% match rate while Bowtie matched at 94%.  For BWA the modified parameters are able to change the minimum seed length (mem -k), band width (mem -w), penalties for gaps or mismatches (mem -B, -O), as well as many other facets of these parameters. 
Bowtie shares many similar parameters, both for building the index and the matching algorithm. For our best performance, we created a small index and ran the aligner with the with the --sensitive preset option. We found these preset options (4 total: very-fast, fast, sensitive, very-sensitive) a helpful addition to the aligner as it allowed for easy modification of the algorithm without having to tune each individual parameter.

## Q4:
Code at the bottom. For full table, check the gc_read_table.txt file. Due to the size of our genome, including the full table here was not possible. 

## Q5:
Using Tablet, there seemed to be negligible spikes in reads that start in G/C rich areas.

## Q6:
We simply calculated the Pearson coefficient and coefficient of determination for the entire genome. Additionally, we calculated the Pearson coefficient for each individual chromosome. 
Overall there seems to be little correlation between GC-content and read-counts over the entire genome. The global r2 score is at -0.0599 and the global pearson coefficient is at -0.0596. However, there are chromosomes that show more correlation than that. For a detailed more detailed exploration, refer to Figure 1 (below). 
These results are all gathered on the data aligned with bowtie2. 

![Figure1](./gc_coverage_plot.jpg "Figure 1")


## Q7:

# Appendix1: Code

In [1]:
import re
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from sklearn.metrics import r2_score
from tqdm import tqdm_notebook as tqdm

### Q6:

In [2]:
def _count_read_starts(alignment_file):
    read_counts = {}
    
    with open(alignment_file, 'r') as reads:
        for line in tqdm(reads, desc='Read Coverage'):
            # ignores first few lines:
            if not line.startswith('@'):
                fields = line.split('\t')  # parses SAM format columns
                cigar = fields[5]  # sixth field/column contains cigar string
                chromosome = fields[2]  # what chromosome the read is aligned to
                
                # if this is the first occurence on this genome, create a table
                if not chromosome in read_counts.keys() and chromosome != '*':  
                    read_counts[chromosome] = {}
                    
                if cigar == '151M' and chromosome != '*':  # only for perfect alignment
                    r_start = int(fields[3])-1  # 4th field contains 1- based leftmost mapping position
                    if r_start>0:  # if POS column is 0, read is not mapped
                        try:
                            read_counts[chromosome][r_start//100] += 1  
                        except Exception:
                            read_counts[chromosome][r_start//100] = 1  # creates table if this is first read on this segment

    return read_counts

def _parse_genome(g_file):
    chromosomes = {}
    seg_string = ''
    s_idx = -1

    with open(g_file, 'r') as genome:
        for line in tqdm(genome, desc='GC Content'):
            if line.startswith('>'):        
                if len(seg_string)>0:
                    s_idx += 1
                    chromosomes[c_temp][s_idx] = _count_helper(seg_string)
                c_temp = line.split(' ')[0].replace('>', '')
                chromosomes[c_temp] = {}
                s_idx = -1
                
            if not line.startswith('>'):
                seg_string += line.strip()
                if len(seg_string)>100:
                    s_idx += 1
                    chromosomes[c_temp][s_idx] = _count_helper(seg_string[:100])
                    seg_string = seg_string[100:]
        
    return chromosomes    

def _count_helper(segment):
    # used to calculate GC content for given base string
    segment = segment.lower()
    counts = {'a':0,
              't':0,
              'c':0,
              'g':0}
    for letter in segment:
        counts[letter] += 1
        
    return (counts['c']+counts['g'])/sum(counts.values())  # sum(G+C) / sum(A+C+T+G)

def get_corr_scores(sam_file, genome_file, return_dicts=True, plot_name=None):
    # to plot the results and save them, specify plot name
    read_counts = _count_read_starts(sam_file)  
    gc_contents = _parse_genome(genome_file)
    x = []
    y = []
    pearson_vals = []

    tmpx = []
    tmpy = []

    loc_table = {}
    chrom_idx = {}

    cum_seg = 0

    for c_idx, chrom in enumerate(gc_contents.keys()):  # iterate over chromosomes
        for s_idx, seg in enumerate(gc_contents[chrom].keys()):  # iterate over 100-base segments in chromosome
            if s_idx == 0:
                chrom_idx[chrom] = cum_seg
            if s_idx == len(gc_contents[chrom].keys()):
                cum_seg += seg
            tmpx.append(gc_contents[chrom][seg])  
            x.append(gc_contents[chrom][seg])
            try:
                tmpy.append(read_counts[chrom][seg])
                y.append(read_counts[chrom][seg])
                loc_table[cum_seg] = [gc_contents[chrom][seg], read_counts[chrom][seg]] 
            except Exception as e:
                assert isinstance(e, KeyError)
                tmpy.append(0)
                y.append(0)
                loc_table[cum_seg] = [gc_contents[chrom][seg], 0] 
            cum_seg += 1
    #     print(chrom)
    #     print(pearsonr(tmpy,tmpx))
        pearson_vals.append(pearsonr(tmpy,tmpx)[0])
    #     plt.scatter(x,y)
    #     plt.show()
        tmpx = []
        tmpy = []
    # print(cum_seg)
    # print(chrom_idx)
    x = np.array(x)  # gc proportion  
    y = np.array(y)  # coverage
    
    print('Global r2 score:\t\t',r2_score(y, x))
    print('Global person coefficient:\t',pearsonr(x,y)[0])
    
    if plot_name is not None:
        fig, ax1 = plt.subplots(figsize=(20,15))
        gen_locs = np.array([loc for loc in loc_table.keys()])
        cont_read_array = np.array([np.array(val) for val in loc_table.values()])
        gc_conts = cont_read_array[:,0]
        read_cov = cont_read_array[:,1]

        ax2c = 'darkgrey'
        ax1.vlines(chrom_idx.values(), ymin=0, ymax=np.max(read_cov), color=ax2c, alpha=0.5)

        for idx, item in enumerate(chrom_idx.items()):
            ax1.text(item[1]+400, 5000, f'{item[0]}', rotation=90, 
                     verticalalignment='center', color=ax2c)

        im = ax1.scatter(x=gen_locs, y=read_cov, c=gc_conts, cmap='YlOrRd', s=5, alpha=0.95)
        ax1.set_xlabel('Location on the genome (100-base steps)')
        ax1.set_ylabel('Read Coverage')

        ax2 = ax1.twinx()
        ax2.set_ylabel('Pearson Coefficient', color=ax2c)
        ax2.plot([v for v in chrom_idx.values()], pearson_vals, color=ax2c)
        ax2.set_ylim([None,1])
        ax2.tick_params(axis='y', labelcolor=ax2c)

        cbar = fig.colorbar(im, orientation='horizontal')
        cbar.set_label('GC Content')
        plt.savefig('./gc_coverage_plot.jpg', dpi=400, bbox_inches='tight')
        plt.show()
    if return_dicts:
        return [read_counts, gc_contents]

In [3]:
read_counts, gc_contents = get_corr_scores('./data/bowtie2_read_aligns_1.sam', './data/GCF_000146045.2_R64_genomic.fna')


Global r2 score:		 -0.059946737754323465
Global person coefficient:	 -0.05958676949785895


### Q4:

In [8]:
col1 = 'Chromosome - window number'
col2 = 'Position on chromosome'
col3 = 'Number of reads starting in the window'
col4 = 'GC Content'

with open('./gc_read_table.txt', 'w') as out_table:
    template = f'{col1:<26}\t{col2:<22}\t{col3:<38}\t{col4:<10}\n'
    out_table.write(template)
    for c_idx, chrom in enumerate(gc_contents.keys()):  # iterate over chromosomes
        for s_idx, seg in enumerate(gc_contents[chrom].keys()):  # iterate over 100-base segments in chromosome
            if not s_idx==len(gc_contents[chrom].keys()):
                col1 = f'{chrom} - {seg}'
                col2 = f'{seg*100} - {(seg+1)*100}'
                try:
                    col3 = f'{read_counts[chrom][seg]}'
                except Exception as e:
                    assert isinstance(e, KeyError)
                    col3 = '0'
                col4 = f'{gc_contents[chrom][seg]*100}%'
                template = f'{col1:<26}\t{col2:<22}\t{col3:<38}\t{col4:<11}\n'
                out_table.write(template)
            else:
                col1 = f'{chrom} - {seg}'
                col2 = f'{seg*100} - end'
                try:
                    col3 = f'{read_counts[chrom][seg]}'
                except Exception as e:
                    assert isinstance(e, KeyError)
                    col3 = '0'
                col4 = f'{gc_contents[chrom][seg]*100}%'
                template = f'{col1:<26}\t{col2:<22}\t{col3:<38}\t{col4:<11}\n'
                out_table.write(template)